In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import itertools
import gc
import pickle

In [2]:
class VectorQuantizer(torch.nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = torch.nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        # convert inputs from BCHW -> BHWC
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        input_shape = inputs.shape
        
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
            
        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        
        # Loss
        e_latent_loss = torch.nn.functional.mse_loss(quantized.detach(), inputs)
        q_latent_loss = torch.nn.functional.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        # convert quantized from BHWC -> BCHW
        return loss, quantized.permute(0, 3, 1, 2).contiguous(), perplexity, encodings

In [3]:
class Encoder(torch.nn.Module):
  def __init__(self, n_input_channels, hidden_size, latent_dim):
    super().__init__()
    self.model = torch.nn.Sequential(
        torch.nn.Conv2d(n_input_channels, hidden_size, kernel_size=3, stride=2, padding=1),
        torch.nn.GELU(),
        torch.nn.Conv2d(hidden_size, hidden_size, kernel_size=3),
        torch.nn.GELU(),
        torch.nn.Conv2d(hidden_size, 2*hidden_size, kernel_size=3),
        torch.nn.GELU(),
        torch.nn.Conv2d(2*hidden_size, 2*hidden_size, kernel_size=3, stride=2),
        torch.nn.GELU(),
        #torch.nn.Flatten()
    )

    #self.linear_mean = torch.nn.Linear(2*hidden_size*25, latent_dim)
    #self.linear_logvar = torch.nn.Linear(2*hidden_size*25, latent_dim)
    #self.linear = torch.nn.Linear(2*hidden_size*16, latent_dim)

  def forward(self, x):
    x = self.model(x)
    #x = self.linear(x)
    return x

In [4]:
class Decoder(torch.nn.Module):
  def __init__(self, n_input_channels, hidden_size, latent_dim):
    super().__init__()
    #self.linear = torch.nn.Sequential(torch.nn.Linear(latent_dim, 2 * 16 * hidden_size), torch.nn.GELU())

    self.model = torch.nn.Sequential(
        torch.nn.ConvTranspose2d(2*hidden_size, 2*hidden_size, kernel_size=3, stride=2, padding=1, output_padding=1), 
        torch.nn.GELU(),
        torch.nn.Conv2d(2*hidden_size, 2*hidden_size, kernel_size=3, padding=1),
        torch.nn.GELU(),
        torch.nn.ConvTranspose2d(2*hidden_size, hidden_size, kernel_size=3, stride=2, output_padding=1, padding=1), 
        torch.nn.GELU(),
        torch.nn.Conv2d(hidden_size, hidden_size, kernel_size=3), # , padding=1
        torch.nn.GELU(),
        torch.nn.ConvTranspose2d(hidden_size, n_input_channels, kernel_size=3, stride=2, output_padding=1, padding=1),
        torch.nn.Tanh(),
        #torch.nn.Sigmoid(),
    )

  def forward(self, x):
    #x = self.linear(x)
    #x = x.reshape(x.shape[0], -1, 4, 4)
    x = self.model(x)
    return x

In [5]:
class Autoencoder(torch.nn.Module):
  def __init__(self, n_input_channels, hidden_size, latent_dim, num_embeddings, embedding_dim, commitment_cost):
    super().__init__()

    self.encoder = Encoder(n_input_channels, hidden_size, latent_dim)
    self.decoder = Decoder(n_input_channels, hidden_size, latent_dim)
    self._vq_vae = VectorQuantizer(num_embeddings, embedding_dim, commitment_cost)

  def forward(self, x):
    x = self.encoder(x)
    loss, quantized, perplexity, enc = self._vq_vae(x)
    x = self.decoder(quantized)
    return x, enc, loss, perplexity

In [6]:
def visualize_grid(x_batch):
  im_rec = Image.fromarray(torchvision.utils.make_grid((x_batch*0.5+0.5) * 255).permute(1, 2, 0).cpu().numpy().astype(np.uint8))
  #im_rec = Image.fromarray(torchvision.utils.make_grid((x_batch) * 255).permute(1, 2, 0).cpu().numpy().astype(np.uint8))
  return im_rec.resize((im_rec.size[0]*4, im_rec.size[1]*4))

In [7]:
# None-tuned perameters
batch_size = 128
shuffle_train = False

In [8]:
transform = transforms.Compose(
    [
    torchvision.transforms.Resize((28, 28)),
    transforms.ToTensor(),
     #torchvision.transforms.ConvertImageDtype(dtype=torch.uint8),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

transform = transforms.Compose(
    [
    torchvision.transforms.Resize((28, 28)),
    #transforms.RandomHorizontalFlip(0.2),
    #transforms.RandomRotation(10),
    #transforms.RandomEqualize(.5),
    transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5)),
     ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=shuffle_train, num_workers=8)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [9]:
if type(trainset.data) is np.ndarray:
    data_variance = np.var(trainset.data / 255.0)
else:
    data_variance = torch.var(trainset.data / 255.0)

In [10]:

"""n_input_channels, hidden_size, latent_dim = 1, 28, 24
num_embeddings, embedding_dim, commitment_cost = 128, 56, 0.25 # embedding_dim = num_channels at the output of the encoder
learning_rate = 1e-3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Autoencoder(n_input_channels, hidden_size, latent_dim, num_embeddings, embedding_dim, commitment_cost).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)"""
if trainset.data.shape[-1] == 3: 
    n_input_channels, hidden_size, latent_dim = 3, 28, 24
else:
    n_input_channels, hidden_size, latent_dim = 1, 28, 24
    
num_embeddings, embedding_dim, commitment_cost = 128, 56, 0.25 # embedding_dim = num_channels at the output of the encoder
learning_rate = 1e-3

trials = {}
for num_embeddings in range(5, 350, 5): 
    for embedding_dem in range(5, 350, 5): 
        trials[num_embeddings,embedding_dim] = []


# for num_embeddings,embedding_dim in trials.keys():
#     print(num_embeddings,embedding_dim)
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     model = Autoencoder(n_input_channels, hidden_size, latent_dim, num_embeddings, embedding_dim, commitment_cost).to(device)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

#     for epoch in range(50):  # loop over the dataset multiple times
#         running_loss = 0.0
#         for i, data in enumerate(trainloader, 0):
#             # get the inputs; data is a list of [inputs, labels]
#             inputs, labels = data
#             inputs = inputs.to(device)
#             # zero the parameter gradients
#             optimizer.zero_grad()
#             # forward + backward + optimize
#             outputs, encodings, loss, perplexity = model(inputs)
#             loss.backward()
#             optimizer.step()
#             # print statistics
#             running_loss += loss.item()
#             if i % 100 == 99:    # print every 100 mini-batches
#                 print('[%d, %5d] loss: %.3f' %
#                       (epoch + 1, i + 1, running_loss / 100))
#                 running_loss = 0.0
#         trials[num_embeddings,embedding_dim].append(loss.item())
#         print("Epoch", epoch, "loss", loss.item())

#     print('Finished Training')

#     for i, data in enumerate(testloader, 0):
#         inputs, labels = data
#         inputs = inputs.to(device)
#         outputs, encodings, loss, perplexity = model(inputs)
#         im_rec = visualize_grid(outputs)
#         im_rec.save("test.png")
#         break

#     print("Perplexity", perplexity.item())

#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
#         inputs = inputs.to(device)
#         outputs, encodings, loss, perplexity = model(inputs)
#         im_rec = visualize_grid(outputs)
#         im_rec.save("train.png")
#         break

#     print("Perplexity", perplexity.item())

#     torch.save(model.state_dict(), "model.pt")

#     #with open("model.pt", "wb") as f:
#     #    torch.save(model, f)

#     #with open("model.pt", "rb") as f:
#     #    model = torch.load(f)


In [11]:

for num_embeddings,embedding_dim in trials.keys():
    learning_rate = 1e-3
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = Autoencoder(n_input_channels, hidden_size, latent_dim, num_embeddings, embedding_dim, commitment_cost).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    for epoch in range(7):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            x, labels = data
            x = x.to(device)
            optimizer.zero_grad()

            x_hat, enc, vq_loss, perplexity = model(x)
            mse_loss = torch.nn.functional.mse_loss(x, x_hat) / data_variance
            #mse_loss = mse_loss.sum(dim=[1, 2, 3]).mean(dim=[0])
            #bce_loss = torch.nn.functional.binary_cross_entropy(x_hat.view(-1, 1024), x.view(-1, 1024), reduction='sum')
            #loss = bce_loss + kl_loss 
            loss = mse_loss + vq_loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % batch_size * 5 == batch_size * 5 - 5:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / (batch_size * 5):.3f}')
                running_loss = 0.0

        if (epoch + 1) % 3 == 0:
          for g in optimizer.param_groups:
            learning_rate *= 0.1
            g['lr'] = learning_rate
    trials[num_embeddings,embedding_dim].append(running_loss / (batch_size * 5))
    gc.collect()
with open('tune_hyperperameters.pkl', 'bw') as f:
        pickle.dump(trials, f)
print(trials)

RuntimeError: shape '[-1, 5]' is invalid for input of size 114688

In [ ]:
with open('tune_hyperperameters.txt', 'w') as f:
    for key, value in trials.items():
        f.write(f'num_embeddings:{key[0]} - embedding_dim:{key[1]} - loss:{value}\n')

In [ ]:
trials.items()

In [ ]:
for data in testloader:
  x, labels = data
  x = x.to(device)

  with torch.no_grad():
    x_hat = model(x)[0]

  break

In [ ]:

# with open('tune_hyperperameters.pkl', 'br') as f:
#     trials = pickle.load(f)


In [ ]:
visualize_grid(x_hat)

In [ ]:
visualize_grid(x)

In [ ]:
means, lbls = [], []
for data in testloader:
  x, labels = data
  x = x.to(device)

  with torch.no_grad():
    x_mean = model.encoder(x)[0]
  means.append(x_mean)
  lbls.append(labels)

features = torch.cat(means,0)
features = features.detach().cpu().numpy()

nsamples, nx, ny = features.shape
features = features.reshape((nsamples,nx*ny))

labels = torch.cat(lbls).numpy()

print(features.shape, labels.shape)

tsne = TSNE(n_components=2,learning_rate='auto',init='pca',perplexity=30).fit_transform(features) #,

In [ ]:
colors = np.array(["red","green","blue","yellow","pink","black","orange","purple","beige","brown"])
c = np.array([colors[el] for el in labels])
tsne_sel = tsne#[(labels==1)|(labels==4)]
#col_sel = c#[(labels==3)|(labels==5)]
#plt.scatter(tsne_sel[:,0], tsne_sel[:,1], c=col_sel)
plt.scatter(tsne_sel[:,0], tsne_sel[:,1]) #,c=col_sel)